In [15]:
import os
import sys
import uuid

pipeline_dir = os.path.join(os.getcwd(), '..')
if pipeline_dir not in sys.path:
    sys.path.append(pipeline_dir)

In [3]:
from pipeline import Pipeline, events, resources

# Getting Started
You can create a new project with the `pipeline-create <new_directory>` which generates a template project in a new directory.  Edit the `handler.py` file to build your pipeline.

# Building a Pipeline
Pipelines are made up of lambda functions, events, resources, and services.  Let us start by defining an empty pipeline by inheriting from the base class.

In [20]:
class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline")

## Define a Function
We can define a lambda function by defining a class method decorated with an event trigger.  The decorator determines how the lambda function is executed.  Let's write a simple "Hello World" function triggered with a GET request.

In [17]:
class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline")
    
    @events.http(method="get", path="helloworld", cors="true")
    def hello_world(self, event, context):
        print("Hello World!")

## Define a Resource
All that is required to build a working pipeline is a pipeline and at least one lambda function with an event trigger.  If we want our pipeline to interact with other AWS resources we may define a resource.  Let's define a DynamoDB table which we will use to store messages:

In [10]:
class MyTable(resources.DynamoDB):
    
    def __init__(self):
        super().__init__()
        self.add_attribute('id', 'S')
        self.add_key('id', 'HASH')

We will also edit our original lam|bda function to take user input in the GET request.  We can register our newly created resource by instantiating and passing into the call to `super`.  Additionally, resources provide helper methods for interacting with the resource.  The DynamoDB resource, for example, inherits the `put`, `get`, `delete`, and `list` methods.

In [16]:
table = MyTable()

class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline",
                         resources=[table])
    
    @events.http(method="get", path="helloworld/{message}", cors="true")
    def hello_world(self, event, context):
        item = {'id': str(uuid.uuid1),
                'message': event['message']
               }
        # Use helper method
        table.put(item)

## Packaging
Now that your pipeline is built, we need to do some final packaging before we can deploy.  Because lambda can't call functions inside of classes, we need to add wrappers for each function handler.  Additionally, we need to add a deploy wrapper for deploying the pipeline.  Once finished your project should look something like `cognition-pipeline/examples/quickstart/`.

In [19]:
#Instantiate your pipeline
pipeline = MyFirstPipeline()

def hello_world(event, context):
    pipeline.hello_world(event, context)

def deploy():
    pipeline.deploy()

## Requirements
Requirements may either be added to `requirements.txt` or handled through the use of services.

## Deploying
Once your pipeline is packaged it is time to deploy!  Use the `pipeline-deploy <pipeline_directory>` to generate a `serverless.yml` file which may be deployed with Serverless Framework.

## Testing
Once deployed, send a GET request to your function's endpoint with a customized message.  You can now go into the AWS console and find your message in your DynamoDB table.  Better yet, you could write a second lambda function to return all messages in the table:

In [22]:
class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline",
                         resources=[table])
    
    @events.http(method="get", path="helloworld/{message}", cors="true")
    def hello_world(self, event, context):
        item = {'id': str(uuid.uuid1),
                'message': event['message']
               }
        # Use helper method
        table.put(item)
    
    @events.http(method="get", path="helloworld", cors="true")
    def list_messages(self, event, context):
        messages = table.list()
        resp = {'statusCode': 200,
                'body': json.dumps(messages)}
        return messages

pipeline = MyFirstPipeline()

def hello_world(event, context):
    pipeline.hello_world(event, context)

def list_messages(event, context):
    resp = pipeline.list_messages(event, context)
    return resp

def deploy():
    pipeline.deploy()